# Milestone1



# Introduction

Conway's Game of Life is a cellular automata used to study complex patterns from arise a set of rules. In this report I will explore how different methods for optimisation and parallelization can be used to increase the efficiency of the the computation used in this cellular automata. To begin with a serial implementation will be completed and then run at different levels of compiler optimisation, then followed by an parallelized implementation and an analysis of the difference between the two.

## Conway's Game of Life

Cellular automata is a model used to simulate and study self-repuducing systems in an environment. This allows for many real world applications such as, the study of viral infection dynamics, lattice gas simulation and other dynamic systems in science, technology and mathematics. For this analysis Conway's Game of Life will be used as the environment, which follows the following rules:
> 1. Any live call with fewer than 2 neighbors dies = underpopulation
> 2. Any live cell with two or three live neighbors lives on to the next gen
> 3. Any live cell with more than 3 live neighbors dies = overpopulation
> 4. Any dead cell with exactly 3 live neighbors becomes living = reproduction

This version of the cellular automata is a zero-player game, which means the inital state of the simulation will determine how the evolution of everything within the environment will occur. The inital state normally contains common patterns, for example, oscillators, which are patterns that return to their initial state after a few generations or spaceships that fly across the grid. Combining these simple inital state patterns is how you can get bigger more complex behaviors. Conway's Game of Life is also Turing complete meaning that it can be used to simulate a turning machine.

The key piece for his model is how the grid is represented, as this will change the efficiency of the model and how all the data can be processed. The most common way for the grid to be created is using a 2D array, and having the bounds of the grid all count as dead cells. Doing the simulation this way will lead to memory issues with bigger simulations and also inaccuracy of some patterns as hitting the side of the grid will just kill off some cells, but the pattern will still run, just with another behavior. In theory, the patterns should be infinite so the grid should be wrapped around allowing cells to continue at the opposite end of the grid when a border is hit, so when studying the behavior of the patterns alone, this functionality of the grid should be added for the most accuracy.

# Algorithm and current Serial Implementation
For this project the Game of Life was constructed in a way that would allow for ease of implemenation but still provide correct the correct results. Since the point of this project is analysis the difference between serial vs parallel implementations of a program, the behavior of the patterns did not have to have infinite grid space. This allowed for the grid to be implemented in the most common way of having all the cells outside the defined grid classified as dead. For this projects implemenation of the grid a conventional 2D array was not used, instead a 1D C++ vector was used to simulated a 2D array, this allowed for faster indexing then what a 2D implemenation allows, there by increasing the efficiency of the program. Currently the implementation for checking neighbors is done by indexing around a particular cell in a loop iteration, which has lead to a lot of if statements within the aliveNeighbors function. This was done in the interest of ease for the first implementation and is an ideal place for improvement for the final demonstration. The current change that is being suggested will be to change it to a form of iteration around a point using mod arithmatic or using a convolve method with a Game of Life kernel which is 3*3 and all 1's with a 0 as the center point.

## Verification

To do verification for the Game of Life simple known patterns had to be used so that the behaviors could be observed. This lead to the creation of the displayGoL.py script in the Serial/ file within the project. displayGoL.py is a script that will take commandline arguments and pass them to ./main to be run, and if the command Vis is passed then a visualization of the patterns behavior will occur. Using this created tool I was able to compare simple patterns such a glider, block, blinker and glider gun to known working implemenations of GoL, like "Golly" that is found of the ConwayLife.com website. Using the same patterns same pattern as what I did on my own implementation and comparing their behaviors were the same, it was found that my implemenation does infact work with the GoL rules/environment. The only difference was when a boundry is hit, the behavior diverges from that of a infinite grid, and as discussed above this is fine for this use case of the environment.

# Results of Milestone 1 Serial Implementation

## Data Results

> The following data will use these abbreviations to save space:
> * Gosper_glider_gun(Gospe)
> * Period144Oscillator(Perio)
> * Frothing_puffer(Froth) 
> * 13enginecordership(13eng)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np 
import json

testData = {}
with open("data.json", 'r') as jFile:
    testData = json.load(jFile)

In [ ]:

for opti, optival in testData.items():
    for gen, genval in optival.items():
        newFig = plt.figure(figsize=(20,3), dpi=200)
        plt.title("OptimiseFlag:" + opti + "    Generations: "  + gen, fontdict={'fontsize':10})
        plt.xlabel("Pattern|Grid Size")
        plt.ylabel("Average Time(s)")
        for grid, gridval in genval.items():
            for pat, patval in gridval.items():
                plt.bar(pat[:5] + " "  + grid + "^2", np.average(patval),yerr=np.std(patval), align='center', width=0.5)
                if "Period144" in pat:
                    newFig.savefig("OptimiseFlag:" + opti + "    Generations: "  + gen + ".png")
                



## Profiling Results

> Below is a screenshot of the top of my profile results showing potential      areas for optimisation

![](Milestone1Profile.png)

> This is the legend given by the profiling tool gprof and all the below key is from their output:

* % time: The percentage of the total running time of the program used by this       function.
* cumulative seconds: A running sum of the number of seconds accounted for by this        function and those listed above it.
* self seconds: The number of seconds accounted for by this function alone. This is       the major sort for this listing.
* calls: The number of times this function was invoked, if this function is profiled,     else blank.
* self ms/call: The average number of milliseconds spent in this function per call,       if this function is profiled, else blank.
    total ms/call: The average number of milliseconds spent in this function and        its descendents per call, if this function is profiled, else blank.
* name: The name of the function.  This is the minor sort for this listing. The         index shows the location of the function in the gprof listing. If the index is      inparenthes is it shows where it would appear in the gprof listing if it were       to be printed.






## Results Discussion

### Data Analysis

From the graphs above the effects of compiler optimization, grid size and pattern size on the program can be seen very clearly. This data was gathered by taking the average of 4 different executions each time for each pattern, grid size, optimisation level and amount of generations. The 4 patterns used from smallest to largest were Gosper_glider_gun(Gospe), Period144Oscillator(Perio), Frothing_puffer(Froth), 13enginecordership(13eng). From the results gathered, the standard deviation was also taken and used within an error bar on all the graphs, depicting that every test had similar results with the acception of "Gospe 1024^2" in the levelTwo/10000 graph. Even with this variation with the runs, the average was still inline with the trend of the rest of the of the data. 

#### No optimisation
In the graphs with no optimisation it can be seen that the amount of generations that the environment simulates will dramatically increase the runtime of the simulation. Just going from 1000 to 10000 generations took the simulation around 10 times longer to complete. The results found also suggest that the grid size also dramatically affects runtime of the program, as for each time the grid size paramater was doubled, the runtime was increase by approximatly 3.5 times. From this the finds do suggest that the size of the pattern does not actually matter to the runtime of the program, since all the patterns finished around the same time and runtime was only ever increased with an increase of grid size or generations ran.

#### Compiler Optimisation

With compiler optimisation the run times of the program decreases all decreased by a factor of about 7 in every case. Between the 3 levels of compiler optimisation that g++ allows there was not much much improvement and for the case of level two compile optimisation, got slightly worse then its level one counter part. Even with the optimisations the times were still there still was a huge jump in runtimes every time the grid size was doubled. Even in the optimisation tests, pattern size made no difference to runtime. But these findings do prove that having some form of compiler optimisation will make the runtime of the game of life simulation faster.


### Profiling Analysis

Ouput from gprof shows that there was 2 calls that took up the most time while the program was running, these where Grid::aliveNeighbors and Vector2D::index. To get these finding gprof was run using the Frothing_puffer pattern on a 1024^2 grid for 10000 generations. The time that the program was hung on these function calls was far more significant then any other calling, leading me to believe there could be some optimisation done on these functions, that isn't just done by the compiler. For Vector2D::index, the function could be sped up by changing the underlying implementation from using a vector<int> to a normal array of size = width*height. This will allow for faster lookups in memory since everything in an array is next to each other, rather then being spread out like a vector<int>. In the case of Grid::aliveNeighbors, I could clean up the function so it had less control flow, this would allow for variable initialization saving memory on each iteration. This could be done by making a convolve method that uses a game of life kernal to check the neighbors of the cell in question. Doing this would be far less beneficial as it wouldn't save to much memory any may introduce more bugs, making the program not valid. 


# References